In [98]:
import liionpack as lp
import numpy as np
import pandas as pd
import pybamm

In [66]:
model = pybamm.lithium_ion.DFN()

In [67]:
params = pybamm.ParameterValues("Chen2020")

In [68]:
sim = pybamm.Simulation(model, parameter_values=params)

In [69]:
sim.solve([0,3600])

In [ ]:
# Generate the netlist
netlist = lp.setup_circuit(Np=16, Ns=2, Rb=1e-4, Rc=1e-2, Ri=5e-2, V=3.2, I=80.0)
lp.draw_circuit(netlist, cpt_size=1.0, dpi=200, node_spacing=2.5, scale=0.75)

In [ ]:

output_variables = [
    'X-averaged total heating [W.m-3]',
    'Volume-averaged cell temperature [K]',
    'X-averaged negative particle surface concentration [mol.m-3]',
    'X-averaged positive particle surface concentration [mol.m-3]',
]

# Heat transfer coefficients
htc = np.ones(32) * 10

# Cycling experiment, using PyBaMM
experiment = pybamm.Experiment([
    "Charge at 20 A for 30 minutes",
    "Rest for 15 minutes",
    "Discharge at 20 A for 30 minutes",
    "Rest for 30 minutes"],
    period="10 seconds")

# PyBaMM parameters
parameter_values = pybamm.ParameterValues("Chen2020")

# Solve pack
output = lp.solve(netlist=netlist,
                  parameter_values=parameter_values,
                  experiment=experiment,
                  output_variables=output_variables,
                  htc=htc)

In [167]:
# Generate the netlist
netlist = lp.setup_circuit(Np=4, Ns=1, Rb=1e-3, Rc=1e-2)
# Define some additional variables to output
output_variables = [
    'X-averaged total heating [W.m-3]',
    'Volume-averaged cell temperature [K]',
    'X-averaged negative particle surface concentration [mol.m-3]',
    'X-averaged positive particle surface concentration [mol.m-3]',
]

# Cycling experiment, using PyBaMM
# experiment = pybamm.Experiment([
# "Charge at 5 A for 30 minutes",
# "Rest for 15 minutes",
# "Discharge at 5 A for 30 minutes",
# "Rest for 30 minutes"],
# period="10 seconds"),

experiment = pybamm.Experiment(
    [
        (
            "Charge at 1.5 A for 1000 s or until 4.2 V",
            "Rest for 1000 s",
            "Discharge at 1 A for 1000 s or until 2.7 V",
            "Rest for 1000 s",
        )
    ],
    period="10 s",
)

# Heat transfer coefficients
inputs = {"Total heat transfer coefficient [W.m-2.K-1]": np.ones(4) * 10.0}

# PyBaMM parameters
parameter_values = pybamm.ParameterValues("Chen2020")

# Solve the pack problem
output = lp.solve(netlist=netlist,
parameter_values=parameter_values,
experiment=experiment,
sim_func=lp.thermal_simulation,
output_variables=output_variables,
initial_soc=0.5,
inputs=inputs)

Stepping simulation:  25%|██████████████▊                                            | 101/401 [00:01<00:05, 55.69it/s]Event: Minimum voltage [V], Batteries: [0, 1]
Event: Minimum voltage [V], Batteries: [0]
Stepping simulation: 100%|███████████████████████████████████████████████████████████| 401/401 [00:07<00:00, 51.10it/s]


In [168]:
output

{'Time [s]': array([   0.,   10.,   20.,   30.,   40.,   50.,   60.,   70.,   80.,
          90.,  100.,  110.,  120.,  130.,  140.,  150.,  160.,  170.,
         180.,  190.,  200.,  210.,  220.,  230.,  240.,  250.,  260.,
         270.,  280.,  290.,  300.,  310.,  320.,  330.,  340.,  350.,
         360.,  370.,  380.,  390.,  400.,  410.,  420.,  430.,  440.,
         450.,  460.,  470.,  480.,  490.,  500.,  510.,  520.,  530.,
         540.,  550.,  560.,  570.,  580.,  590.,  600.,  610.,  620.,
         630.,  640.,  650.,  660.,  670.,  680.,  690.,  700.,  710.,
         720.,  730.,  740.,  750.,  760.,  770.,  780.,  790.,  800.,
         810.,  820.,  830.,  840.,  850.,  860.,  870.,  880.,  890.,
         900.,  910.,  920.,  930.,  940.,  950.,  960.,  970.,  980.,
         990., 1000., 1010., 1020., 1030., 1040., 1050., 1060., 1070.,
        1080., 1090., 1100., 1110., 1120., 1130., 1140., 1150., 1160.,
        1170., 1180., 1190., 1200., 1210., 1220., 1230., 1240., 1

In [206]:
# 초기 딕셔너리 생성
cell_data = {}

# 셀의 수 (여기에서는 4개)
num_cells = 4

# 각 셀별로 딕셔너리에 데이터 저장
for i in range(num_cells):
    cell_data[f"Cell {i+1}"] = {
        "Voltage_measured": output["Terminal voltage [V]"][:, i],
        "Current_measured": output["Cell current [A]"][:, i],
        "Temperature_measured": output["Volume-averaged cell temperature [K]"][:, i] - 273,
        "Current_load" : output["Pack current [A]"],
        "Voltage_load": output["Pack terminal voltage [V]"],
        "Time" : output["Time [s]"],
    }


In [212]:
# 딕셔너리를 데이터프레임으로 변환
df1 = pd.DataFrame.from_dict(cell_data['Cell 3'], orient='index')
df1 = df1.T
df1 = pd.DataFrame.from_dict(cell_data['Cell 3'], orient='index')
df1 = df1.T


In [211]:
df.to_csv("셀데이터 1.csv", index=False, encoding='utf-8-sig')

In [213]:
df

,Voltage_measured,Current_measured,Temperature_measured,Current_load,Voltage_load,Time
0,3.762685,-0.382793,25.149994,-1.5,3.768504,0.0
1,3.764802,-0.381890,25.151245,-1.5,3.770766,10.0
2,3.766262,-0.381388,25.152557,-1.5,3.772243,20.0
3,3.767364,-0.381058,25.153870,-1.5,3.773353,30.0
4,3.768250,-0.380816,25.155212,-1.5,3.774243,40.0
...,...,...,...,...,...,...
396,3.757406,-0.000461,25.163452,0.0,3.757403,3960.0
397,3.757411,-0.000458,25.163269,0.0,3.757409,3970.0
398,3.757416,-0.000455,25.163116,0.0,3.757414,3980.0
399,3.757421,-0.000455,25.162964,0.0,3.757418,3990.0
